In [2]:
!pip install emoji
# For emoji -> text conversion


In [4]:
import pandas as pd
# making my tiny emoji dataset :)
data = {
    "text": [
        "I love this 😍",
        "This is amazing 😂",
        "So beautiful 😊",
        "I am so angry 😡",
        "This is terrible 😠",
        "I feel sad 😢",
        "Not sure about this 😐",
        "Meh 😶",
        "Okay I guess 🙂",
        "What a disaster 💀",
        "Absolutely delighted 😁",
        "Horrible experience 🤮",
        "Pretty good 👍",
        "I don't like it 👎",
        "I'm surprised 😮",
        "This made me cry 😭",
        "So excited 🤩",
        "Bored 😴",
        "That's funny 😆",
        "Yikes 😬",

    ],
    "sentiment": [
        "positive","positive","positive",
        "negative","negative","negative",
        "neutral","neutral","neutral","negative",
        "positive","negative","positive","negative",
        "neutral","negative","positive","neutral",
        "positive","negative"
    ]
}

df = pd.DataFrame(data)
df.head(20)


,text,sentiment
0,I love this 😍,positive
1,This is amazing 😂,positive
2,So beautiful 😊,positive
3,I am so angry 😡,negative
4,This is terrible 😠,negative
5,I feel sad 😢,negative
6,Not sure about this 😐,neutral
7,Meh 😶,neutral
8,Okay I guess 🙂,neutral
9,What a disaster 💀,negative


In [6]:
try:
    from emoji import demojize
    df["text_processed"] = df["text"].apply(lambda t: demojize(t, delimiters=(" ", " ")))
except Exception as e:
    print("emoji package not available")
    df["text_processed"] = df["text"]

df[["text","text_processed","sentiment"]].head(10)


,text,text_processed,sentiment
0,I love this 😍,I love this smiling_face_with_heart-eyes,positive
1,This is amazing 😂,This is amazing face_with_tears_of_joy,positive
2,So beautiful 😊,So beautiful smiling_face_with_smiling_eyes,positive
3,I am so angry 😡,I am so angry enraged_face,negative
4,This is terrible 😠,This is terrible angry_face,negative
5,I feel sad 😢,I feel sad crying_face,negative
6,Not sure about this 😐,Not sure about this neutral_face,neutral
7,Meh 😶,Meh face_without_mouth,neutral
8,Okay I guess 🙂,Okay I guess slightly_smiling_face,neutral
9,What a disaster 💀,What a disaster skull,negative


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

X_raw = df["text_processed"]
y = df["sentiment"]

vectorizer = CountVectorizer(ngram_range=(1,2), min_df=1)
X = vectorizer.fit_transform(X_raw)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
print("Train size:", X_train.shape, "Test size:", X_test.shape)


Train size: (14, 104) Test size: (6, 104)


In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=0))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.16666666666666666

Classification Report:
               precision    recall  f1-score   support

    negative       0.00      0.00      0.00         2
     neutral       0.25      0.50      0.33         2
    positive       0.00      0.00      0.00         2

    accuracy                           0.17         6
   macro avg       0.08      0.17      0.11         6
weighted avg       0.08      0.17      0.11         6

Confusion matrix:
 [[0 2 0]
 [1 1 0]
 [1 1 0]]


In [10]:
#trying custom inputs
examples = [
    "I am so happy 😂😍",
    "This is bad 😡",
    "Not feeling great 😐",
    "Amazing work 👏😊",
    "I am shocked 😮",
    "Gross 🤮"
]


try:
    examples_proc = [demojize(s, delimiters=(" ", " ")) for s in examples]
except:
    examples_proc = examples

X_ex = vectorizer.transform(examples_proc)
preds = model.predict(X_ex)
for text, pred in zip(examples, preds):
    print(f"{text}  →  {pred}")


I am so happy 😂😍  →  positive
This is bad 😡  →  negative
Not feeling great 😐  →  neutral
Amazing work 👏😊  →  positive
I am shocked 😮  →  negative
Gross 🤮  →  negative
